# Rotation CIFAR Experiment

This experiment will use images from the **CIFAR-100** database (https://www.cs.toronto.edu/~kriz/cifar.html) and showcase the backward transfer efficiency of algorithms in the **Progressive Learning** project (https://github.com/neurodata/progressive-learning) as the images are rotated.

In [1]:
# Import the packages for experiment
import warnings
warnings.simplefilter("ignore")

import matplotlib.pyplot as plt
import random
import pickle
from skimage.transform import rotate
from scipy import ndimage
from skimage.util import img_as_ubyte
from joblib import Parallel, delayed
from sklearn.ensemble.forest import _generate_unsampled_indices
from sklearn.ensemble.forest import _generate_sample_indices
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from itertools import product
import keras

from keras import layers

from joblib import Parallel, delayed
from multiprocessing import Pool

import tensorflow as tf

from numba import cuda

In [2]:
# Import the progressive learning packages
from proglearn.progressive_learner import ProgressiveLearner
from proglearn.deciders import SimpleAverage
from proglearn.transformers import TreeClassificationTransformer, NeuralClassificationTransformer 
from proglearn.voters import TreeClassificationVoter, KNNClassificationVoter

In [3]:
# Randomized selection of training and testing subsets
def cross_val_data(data_x, data_y, total_cls=10):
    x = data_x.copy()
    y = data_y.copy()
    idx = [np.where(data_y == u)[0] for u in np.unique(data_y)]
    
    
    for i in range(total_cls):
        indx = idx[i]#np.roll(idx[i],(cv-1)*500)
        random.shuffle(indx)
        
        if i==0:
            train_x1 = x[indx[0:250],:]
            train_x2 = x[indx[250:500],:]
            train_y1 = y[indx[0:250]]
            train_y2 = y[indx[250:500]]
            
            test_x = x[indx[500:600],:]
            test_y = y[indx[500:600]]
        else:
            train_x1 = np.concatenate((train_x1, x[indx[0:250],:]), axis=0)
            train_x2 = np.concatenate((train_x2, x[indx[250:500],:]), axis=0)
            train_y1 = np.concatenate((train_y1, y[indx[0:250]]), axis=0)
            train_y2 = np.concatenate((train_y2, y[indx[250:500]]), axis=0)
            
            test_x = np.concatenate((test_x, x[indx[500:600],:]), axis=0)
            test_y = np.concatenate((test_y, y[indx[500:600]]), axis=0)
        
    return train_x1, train_y1, train_x2, train_y2, test_x, test_y 

# Algorithms

The progressive-learning repo contains two main algorithms, **Lifelong Learning Forests** (L2F) and **Lifelong Learning Network** (L2N), the difference being that L2F uses the Uncertainty Forest transformer while L2N uses deep neural networks. Both algorithms, unlike LwF, EWC, Online_EWC, and SI, have been shown to achieve both forward and backward knowledge transfer. Either algorithm can be chosen for the purpose of this experiment.

# Experiment

If the chosen algorithm is trained on both straight up-and-down CIFAR images and rotated CIFAR images, rather than just straight up-and-down CIFAR images, will it perform better (achieve a higher backward transfer efficiency) when tested on straight up-and-down CIFAR images? How does the angle at which training images are rotated affect these results?

At a rotation angle of 0 degrees, the rotated images simply provide additional straight up-and-down CIFAR training data, so the backward transfer efficiency at this angle show whether or not the chosen algorithm can even achieve backward knowledge transfer. As the angle of rotation increases, the rotated images become less and less similar to the original dataset, so the backward transfer efficiency should logically decrease, while still being above 1.

In [4]:
# Chooses model to use as transformer
def choose_transformer(train_x1, test_x, test_y, tmp_data):
    
    # Deep Neural Networks model is used as transformer
    if model == "dnn":

        default_transformer_class = NeuralClassificationTransformer

        network = keras.Sequential()
        network.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=np.shape(train_x1)[1:]))
        network.add(layers.BatchNormalization())
        network.add(layers.Conv2D(filters=32, kernel_size=(3, 3), strides = 2, padding = "same", activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Conv2D(filters=64, kernel_size=(3, 3), strides = 2, padding = "same", activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides = 2, padding = "same", activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Conv2D(filters=254, kernel_size=(3, 3), strides = 2, padding = "same", activation='relu'))

        network.add(layers.Flatten())
        network.add(layers.BatchNormalization())
        network.add(layers.Dense(2000, activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Dense(2000, activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Dense(units=10, activation = 'softmax'))

        default_transformer_kwargs = {"network" : network, 
                                      "euclidean_layer_idx" : -2,
                                      "num_classes" : 10,
                                      "optimizer" : keras.optimizers.Adam(3e-4)
                                     }

        default_voter_class = KNNClassificationVoter
        default_voter_kwargs = {"k" : int(np.log2(len(train_x1)))}

        default_decider_class = SimpleAverage

    # Uncertainty Forest model is used as transformer
    elif model == "uf":

        train_x1 = train_x1.reshape((train_x1.shape[0], train_x1.shape[1] * train_x1.shape[2] * train_x1.shape[3]))
        tmp_data = tmp_data.reshape((tmp_data.shape[0], tmp_data.shape[1] * tmp_data.shape[2] * tmp_data.shape[3]))
        test_x = test_x.reshape((test_x.shape[0], test_x.shape[1] * test_x.shape[2] * test_x.shape[3]))

        default_transformer_class = TreeClassificationTransformer
        # Max depth of tree is set
        default_transformer_kwargs = {"kwargs" : {"max_depth" : 5}}

        default_voter_class = TreeClassificationVoter
        default_voter_kwargs = {}

        default_decider_class = SimpleAverage
        
    return (train_x1, test_x, tmp_data, default_transformer_class, default_transformer_kwargs, 
            default_voter_class, default_voter_kwargs, default_decider_class)

In [5]:
# Runs the experiments
def LF_experiment(data_x, data_y, angle, model, granularity, reps=1, ntrees=29, acorn=None):
    if acorn is not None:
        np.random.seed(acorn)
    
    errors = np.zeros(2)
    
    with tf.device('/gpu:'+str(int(angle //  granularity) % 4)):
        for rep in range(reps):
            print("rep:{}".format(rep))
            train_x1, train_y1, train_x2, train_y2, test_x, test_y = cross_val_data(data_x, data_y, total_cls=10)

            # change data angle for second task
            tmp_data = train_x2.copy()
            _tmp_ = np.zeros((32,32,3), dtype=int)
            total_data = tmp_data.shape[0]

            for i in range(total_data):
                tmp_ = image_aug(tmp_data[i],angle)
                tmp_data[i] = tmp_
                
            # Call choose_transformer function
            (train_x1, test_x, tmp_data, default_transformer_class, default_transformer_kwargs, 
             default_voter_class, default_voter_kwargs, default_decider_class) = choose_transformer(train_x1, test_x, 
                                                                                                    test_y, tmp_data)
            
            progressive_learner = ProgressiveLearner(default_transformer_class = default_transformer_class, 
                                         default_transformer_kwargs = default_transformer_kwargs,
                                         default_voter_class = default_voter_class,
                                         default_voter_kwargs = default_voter_kwargs,
                                         default_decider_class = default_decider_class)

            progressive_learner.add_task(
                X = train_x1, 
                y = train_y1,
                # 67% of the data is used for transformers, 33% is used for voters
                transformer_voter_decider_split = [0.67, 0.33, 0],
                decider_kwargs = {"classes" : np.unique(train_y1)}
            )

            progressive_learner.add_transformer(
                X = tmp_data, 
                y = train_y2,
                transformer_data_proportion = 1,
                backward_task_ids = [0]
            )

            llf_task1=progressive_learner.predict(test_x, task_id=0)
            llf_single_task=progressive_learner.predict(test_x, task_id=0, transformer_ids=[0])

            errors[1] = errors[1]+(1 - np.mean(llf_task1 == test_y))
            errors[0] = errors[0]+(1 - np.mean(llf_single_task == test_y))
    
    errors = errors/reps
    print("Errors For Angle {}: {}".format(angle, errors))
    
    # Save pickle file
    with open('results/angle_'+str(angle)+'_'+model+'.pickle', 'wb') as f:
        pickle.dump(errors, f, protocol = 2)

In [6]:
# Rotates the image by the given angle and zooms in to remove unnecessary white space at the corners
def image_aug(pic, angle, centroid_x=23, centroid_y=23, win=16, scale=1.45):
    im_sz = int(np.floor(pic.shape[0]*scale))
    pic_ = np.uint8(np.zeros((im_sz,im_sz,3),dtype=int))
    
    pic_[:,:,0] = ndimage.zoom(pic[:,:,0],scale)
    
    pic_[:,:,1] = ndimage.zoom(pic[:,:,1],scale)
    pic_[:,:,2] = ndimage.zoom(pic[:,:,2],scale)
    
    image_aug = rotate(pic_, angle, resize=False)
    #print(image_aug.shape)
    image_aug_ = image_aug[centroid_x-win:centroid_x+win,centroid_y-win:centroid_y+win,:]
    
    return img_as_ubyte(image_aug_)

# Hyperparameters

Hyperparameters determine how the model will run. Changing the value of `model` to `"uf"` will run the L2F algorithm, while `"dnn"` will run the L2N algorithm.

`granularity` refers to the amount by which the angle will be increased each time. Setting this value at 1 will cause the algorithm to test every whole number rotation angle between 0 and 180 degrees.

`reps` refers to the number of repetitions tested for each angle of rotation. For each repetition, the data is randomly resampled/

In [7]:
### MAIN HYPERPARAMS ###
model = "uf"
granularity = 1
reps = 50
########################

In [8]:
# Loads and reshapes data sets
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
data_x = np.concatenate([X_train, X_test])
data_y = np.concatenate([y_train, y_test])
data_y = data_y[:, 0]

In [9]:
# Runs the experiment at a new angle of rotation
def perform_angle(angle):
    LF_experiment(data_x, data_y, angle, model, granularity, reps=reps, ntrees=16, acorn=1)

In [10]:
# Run L2N
if model == "dnn":
    for angle_adder in range(0, 180, granularity * 4):
        angles = angle_adder + np.arange(0, granularity * 4, granularity)
        with Pool(4) as p:
            p.map(perform_angle, angles)
            
# Run L2F
elif model == "uf":
    angles = np.arange(0, 180, granularity)
    with Pool(8) as p:
        p.map(perform_angle, angles)

rep:0rep:0rep:0rep:0
rep:0



rep:0rep:0

rep:0
rep:1
rep:1
rep:1
rep:1
rep:1
rep:1
rep:1
rep:1
rep:2
rep:2
rep:2
rep:2rep:2

rep:2
rep:2
rep:2
rep:3
rep:3rep:3

rep:3
rep:3
rep:3
rep:3
rep:3
rep:4
rep:4
rep:4
rep:4
rep:4
rep:4
rep:4
rep:4
rep:5
rep:5
rep:5
rep:5
rep:5
rep:5rep:5

rep:5
rep:6
rep:6
rep:6
rep:6
rep:6
rep:6
rep:6
rep:6
rep:7
rep:7
rep:7
rep:7
rep:7
rep:7
rep:7
rep:7
rep:8
rep:8
rep:8
rep:8
rep:8
rep:8
rep:8
rep:8
rep:9
rep:9
rep:9
rep:9
rep:9
rep:9
rep:9
rep:9
rep:10
rep:10
rep:10
rep:10
rep:10
rep:10
rep:10
rep:10
rep:11
rep:11
rep:11
rep:11
rep:11
rep:11
rep:11
rep:11
rep:12
rep:12
rep:12
rep:12
rep:12
rep:12
rep:12
rep:12
rep:13
rep:13
rep:13
rep:13
rep:13
rep:13
rep:13
rep:13
rep:14
rep:14
rep:14
rep:14
rep:14
rep:14
rep:14
rep:14
rep:15
rep:15
rep:15
rep:15
rep:15
rep:15
rep:15
rep:15
rep:16
rep:16
rep:16
rep:16
rep:16
rep:16
rep:16
rep:16
rep:17
rep:17
rep:17
rep:17
rep:17
rep:17
rep:17
rep:17
rep:18
rep:18
rep:18
rep:18
rep:18
rep:18
rep:18
rep:18
rep:19
rep:19
re

rep:39
rep:39
rep:39
rep:39
rep:40
rep:40
rep:40
rep:40
rep:40
rep:40
rep:40
rep:40
rep:41
rep:41
rep:41
rep:41
rep:41
rep:41
rep:41rep:41

rep:42
rep:42
rep:42
rep:42rep:42

rep:42
rep:42
rep:42
rep:43
rep:43
rep:43
rep:43
rep:43
rep:43
rep:43
rep:43
rep:44
rep:44
rep:44
rep:44
rep:44
rep:44
rep:44
rep:44
rep:45
rep:45
rep:45
rep:45
rep:45
rep:45
rep:45
rep:45
rep:46
rep:46
rep:46
rep:46
rep:46
rep:46
rep:46
rep:46
rep:47
rep:47
rep:47
rep:47
rep:47
rep:47
rep:47
rep:47
rep:48
rep:48
rep:48
rep:48
rep:48
rep:48
rep:48
rep:48
rep:49
rep:49
rep:49
rep:49
rep:49
rep:49
rep:49
rep:49
Errors For Angle 2: [0.74964 0.71066]
rep:0
Errors For Angle 8: [0.74592 0.70942]
rep:0
Errors For Angle 26: [0.74894 0.71126]
rep:0
Errors For Angle 38: [0.7457  0.71018]
rep:0
Errors For Angle 14: [0.74938 0.71014]
rep:0
Errors For Angle 32: [0.7495 0.7149]
rep:0
Errors For Angle 44: [0.74942 0.71456]
rep:0
Errors For Angle 20: [0.75234 0.71578]
rep:0
rep:1
rep:1
rep:1
rep:1
rep:1
rep:1
rep:1
rep:1
rep:2
re

rep:23
rep:23
rep:23
rep:23
rep:24
rep:24
rep:24
rep:24
rep:24
rep:24
rep:24
rep:24
rep:25
rep:25
rep:25
rep:25
rep:25
rep:25
rep:25
rep:25
rep:26
rep:26
rep:26
rep:26
rep:26
rep:26
rep:26
rep:26
rep:27
rep:27
rep:27
rep:27
rep:27
rep:27
rep:27
rep:27
rep:28
rep:28
rep:28
rep:28
rep:28
rep:28
rep:28
rep:28
rep:29
rep:29
rep:29
rep:29
rep:29
rep:29
rep:29
rep:29
rep:30
rep:30
rep:30
rep:30
rep:30
rep:30
rep:30
rep:30
rep:31
rep:31
rep:31
rep:31
rep:31
rep:31
rep:31
rep:31
rep:32
rep:32
rep:32
rep:32
rep:32
rep:32
rep:32
rep:32
rep:33
rep:33
rep:33
rep:33
rep:33
rep:33
rep:33
rep:33
rep:34
rep:34
rep:34
rep:34
rep:34
rep:34
rep:34
rep:34
rep:35
rep:35
rep:35
rep:35
rep:35
rep:35
rep:35
rep:35
rep:36
rep:36
rep:36
rep:36
rep:36
rep:36
rep:36
rep:36
rep:37
rep:37
rep:37
rep:37
rep:37
rep:37
rep:37
rep:37
rep:38
rep:38
rep:38
rep:38
rep:38
rep:38
rep:38
rep:38
rep:39
rep:39
rep:39
rep:39
rep:39
rep:39
rep:39
rep:39
rep:40
rep:40
rep:40
rep:40
rep:40
rep:40
rep:40
rep:41
rep:40
rep:41
rep:41

rep:7
rep:7
rep:7
rep:7
rep:7
rep:7
rep:7
rep:7
rep:8
rep:8
rep:8
rep:8
rep:8
rep:8
rep:8
rep:9
rep:8
rep:9
rep:9
rep:9
rep:9
rep:9
rep:9
rep:9
rep:10
rep:10
rep:10
rep:10
rep:10
rep:10
rep:10
rep:10
rep:11
rep:11
rep:11
rep:11
rep:11
rep:11
rep:11
rep:11
rep:12
rep:12
rep:12
rep:12
rep:12
rep:12
rep:12rep:12

rep:13
rep:13
rep:13
rep:13
rep:13
rep:13
rep:13
rep:13
rep:14
rep:14
rep:14
rep:14
rep:14
rep:14
rep:14
rep:14
rep:15
rep:15
rep:15
rep:15
rep:15
rep:15
rep:15
rep:15
rep:16
rep:16
rep:16
rep:16
rep:16
rep:16
rep:16
rep:16
rep:17
rep:17
rep:17
rep:17
rep:17
rep:17
rep:17
rep:17
rep:18
rep:18
rep:18
rep:18
rep:18
rep:18
rep:18
rep:18
rep:19
rep:19
rep:19
rep:19
rep:19
rep:19
rep:19
rep:19
rep:20
rep:20
rep:20
rep:20
rep:20
rep:20
rep:20
rep:20
rep:21
rep:21
rep:21
rep:21
rep:21
rep:21
rep:21
rep:21
rep:22
rep:22
rep:22
rep:22
rep:22
rep:22
rep:22
rep:22
rep:23
rep:23
rep:23
rep:23
rep:23
rep:23
rep:23
rep:23
rep:24
rep:24
rep:24
rep:24
rep:24
rep:24
rep:24
rep:25
rep:24
rep:25
re

rep:45
rep:45
rep:45
rep:45
rep:45
rep:46
rep:46
rep:46
rep:46
rep:46
rep:46
rep:46
rep:47
rep:46
rep:47
rep:47
rep:47
rep:47
rep:47
rep:47
rep:48
rep:47
rep:48
rep:48
rep:48
rep:48
rep:48
rep:48
rep:49
rep:48
rep:49
rep:49
rep:49
rep:49
rep:49
rep:49
Errors For Angle 52: [0.75122 0.71718]
rep:0
rep:49
Errors For Angle 70: [0.7474  0.71366]
rep:0
Errors For Angle 88: [0.75336 0.71824]
rep:0
Errors For Angle 76: [0.75286 0.71872]
rep:0
Errors For Angle 58: [0.7479  0.71368]
rep:0
Errors For Angle 82: [0.74576 0.71388]
rep:0
Errors For Angle 94: [0.74666 0.7188 ]
rep:0
rep:1
Errors For Angle 64: [0.74728 0.71362]
rep:0
rep:1
rep:1
rep:1
rep:1
rep:1
rep:1
rep:2
rep:1
rep:2
rep:2
rep:2
rep:2
rep:2
rep:2
rep:3
rep:2
rep:3
rep:3
rep:3
rep:3
rep:3
rep:3
rep:4
rep:3
rep:4
rep:4
rep:4
rep:4
rep:4
rep:4
rep:5
rep:4
rep:5
rep:5
rep:5
rep:5
rep:5
rep:5
rep:6
rep:5
rep:6
rep:6
rep:6
rep:6
rep:6
rep:7
rep:6
rep:6
rep:7
rep:7
rep:7
rep:7
rep:7
rep:7
rep:8
rep:7
rep:8
rep:8
rep:8
rep:8
rep:8
rep:8
rep

rep:28
rep:29
rep:29
rep:29
rep:29
rep:30
rep:29
rep:30
rep:29
rep:30
rep:30
rep:30
rep:31
rep:30
rep:31
rep:30
rep:30
rep:31
rep:31
rep:32
rep:31
rep:31
rep:32
rep:31
rep:31
rep:32
rep:32
rep:33
rep:32
rep:32
rep:32
rep:33
rep:32
rep:33
rep:33
rep:34
rep:33
rep:33
rep:33
rep:34
rep:33
rep:34
rep:34
rep:34
rep:35
rep:34
rep:34
rep:35
rep:34
rep:35
rep:35
rep:36
rep:35
rep:35
rep:35
rep:36
rep:35
rep:36
rep:36
rep:37
rep:36
rep:36
rep:36
rep:37
rep:36
rep:37
rep:37
rep:38
rep:37
rep:37
rep:37
rep:38
rep:37
rep:38
rep:38
rep:39
rep:38
rep:38
rep:39
rep:38
rep:38
rep:39
rep:39
rep:39
rep:40
rep:39
rep:39
rep:40
rep:39
rep:40
rep:40
rep:41
rep:40
rep:40
rep:41
rep:40
rep:40
rep:41
rep:41
rep:42
rep:41
rep:42
rep:41
rep:41
rep:41
rep:42
rep:42
rep:43
rep:42
rep:43
rep:42
rep:42
rep:42
rep:43
rep:43
rep:44
rep:43
rep:44
rep:43
rep:43
rep:43
rep:44
rep:44
rep:45
rep:44
rep:45
rep:44
rep:44
rep:44
rep:45
rep:45
rep:45
rep:46
rep:46
rep:45
rep:45
rep:45
rep:46
rep:46
rep:46
rep:47
rep:47
rep:46

rep:12
rep:12
rep:12
rep:12
rep:12
rep:13
rep:14
rep:14
rep:13
rep:13
rep:13
rep:13
rep:13
rep:14
rep:15
rep:15
rep:14
rep:14
rep:14
rep:14
rep:14
rep:15
rep:16
rep:16
rep:15
rep:15
rep:15
rep:15
rep:15
rep:16
rep:17
rep:17
rep:16
rep:16
rep:16
rep:16
rep:16
rep:17
rep:18
rep:18
rep:17
rep:17
rep:17
rep:17
rep:17
rep:18
rep:19
rep:19
rep:18
rep:18
rep:18
rep:18
rep:18
rep:19
rep:20
rep:20
rep:19
rep:19
rep:19
rep:19
rep:19
rep:20
rep:21
rep:20
rep:21
rep:20
rep:20
rep:20
rep:20
rep:21
rep:22
rep:22
rep:21
rep:21
rep:21
rep:21
rep:21
rep:22
rep:23
rep:23
rep:22
rep:22
rep:22
rep:22
rep:22
rep:23
rep:24
rep:23
rep:24
rep:23
rep:23
rep:23
rep:23
rep:24
rep:25
rep:25
rep:24
rep:24
rep:24
rep:24
rep:24
rep:25
rep:26
rep:26
rep:25
rep:25
rep:25
rep:25
rep:25
rep:26
rep:27
rep:27
rep:26
rep:26
rep:26
rep:26
rep:26
rep:27
rep:28
rep:27
rep:27
rep:28
rep:27
rep:27
rep:27
rep:28
rep:29
rep:28
rep:29rep:28

rep:28
rep:28
rep:28
rep:29
rep:30
rep:29
rep:30
rep:29
rep:29
rep:29
rep:29
rep:30
rep:31

rep:12
rep:13
rep:13
rep:12
rep:12
rep:12
rep:13
rep:14
rep:14
rep:13
rep:13
rep:13
rep:14
rep:15
rep:15
rep:14
rep:14
rep:14
rep:15
rep:16
rep:16
rep:15
rep:15
rep:15
rep:16
rep:17
rep:17
rep:16
rep:16
rep:16
rep:17
rep:18
rep:18
rep:17
rep:17
rep:17
rep:18
rep:19
rep:19
rep:18
rep:18
rep:18
rep:19
rep:20
rep:20
rep:19
rep:19
rep:19
rep:20
rep:21
rep:21
rep:20
rep:20
rep:20
rep:21
rep:22
rep:22
rep:21
rep:21
rep:21
rep:22
rep:23
rep:23
rep:22
rep:22
rep:22
rep:23
rep:24
rep:24
rep:23
rep:23
rep:23
rep:24
rep:25
rep:25
rep:24
rep:24
rep:24
rep:25
rep:26
rep:26
rep:25
rep:25
rep:25
rep:26
rep:27
rep:27
rep:26
rep:26
rep:26
rep:27
rep:28
rep:28
rep:27
rep:27
rep:27
rep:28
rep:29
rep:29
rep:28
rep:28
rep:28
rep:29
rep:30
rep:30
rep:29
rep:29
rep:29
rep:30
rep:31
rep:31
rep:30
rep:30
rep:30
rep:31
rep:32
rep:32
rep:31
rep:31
rep:31
rep:33
rep:32
rep:33
rep:32
rep:32
rep:32
rep:34
rep:33
rep:34
rep:33
rep:33
rep:33
rep:34
rep:35
rep:35
rep:34
rep:34
rep:34
rep:35
rep:36
rep:36
rep:35
rep:35

rep:44
rep:44
rep:44
rep:44
rep:45
rep:46
rep:45
rep:45
rep:45
rep:45
rep:46
rep:47
rep:46
rep:46
rep:46
rep:46
rep:47
rep:48
rep:47
rep:47
rep:47
rep:48
rep:47
rep:49
rep:48
rep:48
rep:48
rep:49
rep:48
Errors For Angle 148: [0.75126 0.72374]
rep:0
rep:49
rep:49
rep:49
rep:49
Errors For Angle 160: [0.74676 0.71972]
rep:0
rep:1
Errors For Angle 154: [0.74806 0.7197 ]
rep:0
Errors For Angle 178: [0.74598 0.71768]
rep:0
Errors For Angle 172: [0.74916 0.72116]
rep:0
Errors For Angle 166: [0.74994 0.72356]
rep:0
rep:1
rep:2
rep:1
rep:1
rep:1
rep:1
rep:2
rep:3
rep:2
rep:2
rep:2
rep:2
rep:3
rep:4
rep:3
rep:3
rep:3
rep:4
rep:3
rep:5
rep:4
rep:4
rep:4
rep:5
rep:4
rep:6
rep:5
rep:5
rep:5
rep:6
rep:5
rep:7
rep:6
rep:6
rep:6
rep:7
rep:6
rep:8
rep:7
rep:7
rep:8
rep:7
rep:7
rep:9
rep:8
rep:8
rep:9rep:8

rep:8
rep:10
rep:9
rep:9
rep:9
rep:10
rep:9
rep:11
rep:10
rep:10
rep:10
rep:11
rep:10
rep:12
rep:11
rep:11
rep:11
rep:12
rep:13
rep:11
rep:12
rep:12
rep:12
rep:13
rep:14
rep:12
rep:13
rep:13
rep:14
r

# FAQs

### Why am I getting an "out of memory" error?
`Pool(8)` in the previous cell allows for parallel processing, so the number within the parenthesis should be, at max, the number of cores in the device on which this notebook is being run. Even if a warning is produced, the results of the experimented should not be affected.

### Why is this taking so long to run? How can I speed it up to see if I am getting the expected outputs?
Changing the maximum tree depth does not affect the runtime much. Decreasing the value of `reps`, decreasing the value of `num_transformers`, or increasing the value of `granularity` will all decrease runtime at the cost of noisier results.

### How do I visualize these results?
All results should be saved within a results folder in this rotation_cifar folder. There should also be another file called rotation_plot.ipynb that can be run in order to produce the desired plot of backward transfer efficiency against rotation angle. An important thing to note here is that the `granularity` value in this file should match up with the value in the rotation_plot file.